<a href="https://colab.research.google.com/github/stiepan/MLCourse/blob/master/Mnist_DN_Regularization_7_P3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this lab, you should try to implement some of the techniques discussed in the lecture.
Here is a list of reasonable tasks.
 
Easy:
 * L1 or L2 regularization (choose one)
 * momentum, Nesterov's momentum (choose one)

Medium difficulty:
 * Adagrad, RMSProp (choose one)
 * dropout
 * data augmentation (tiny rotatations, up/down-scalings etc.)

Try to test your network to see if these changes improve accuracy. They improve accuracy much more if you increase the layer size, and if you add more layers.

In [0]:
import random
import numpy as np
from torchvision import datasets, transforms

In [0]:
# Let's read the mnist dataset

def load_mnist(path='.'):
    train_set = datasets.MNIST(path, train=True, download=True)
    x_train = train_set.data.numpy()
    _y_train = train_set.targets.numpy()
    
    test_set = datasets.MNIST(path, train=False, download=True)
    x_test = test_set.data.numpy()
    _y_test = test_set.targets.numpy()
    
    x_train = x_train.reshape((x_train.shape[0],28*28)) / 255.
    x_test = x_test.reshape((x_test.shape[0],28*28)) / 255.

    y_train = np.zeros((_y_train.shape[0], 10))
    y_train[np.arange(_y_train.shape[0]), _y_train] = 1
    
    y_test = np.zeros((_y_test.shape[0], 10))
    y_test[np.arange(_y_test.shape[0]), _y_test] = 1

    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_mnist()

In [0]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    # Derivative of the sigmoid
    return sigmoid(z)*(1-sigmoid(z))

In [148]:
# nestrov's momentum + l1 regularization

class Network(object):
    def __init__(self, sizes, momentum=0.):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.momentum = momentum
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
        self.acc_w_grad = [np.zeros_like(w) for w in self.weights]
        self.acc_b_grad = [np.zeros_like(b) for b in self.biases]
    
    @property
    def final_biases(self):
      return [
        b - self.momentum * v 
        for b, v in zip(self.biases, self.acc_b_grad)
      ]

    @property
    def final_weights(self):
      return [
        w - self.momentum * v
        for w, v in zip(self.weights, self.acc_w_grad)
      ]
    
    def feedforward(self, a):
        # Run the network on a batch
        a = a.T
        for b, w in zip(self.final_biases, self.final_weights):
            a = sigmoid(np.matmul(w, a)+b)
        return a

    def l1_reg(self, lmbda):
      ds = [lmbda * np.sign(w) for w in self.weights]
      eliminated = [np.abs(w) <= np.abs(d) for w, d in zip(self.weights, ds)]
      self.weights = [w - d for w, d in zip(self.weights, ds)]
      for w, z in zip(self.weights, eliminated):
        w[z] = 0.

    
    def update_mini_batch(self, mini_batch, eta, lmbda=None):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch which is as in tensorflow API.
        # eta is the learning rate

        seta = eta/len(mini_batch[0])
        if lmbda is not None:
          self.l1_reg(seta * lmbda)
        
        acc_w_grad = self.acc_w_grad
        acc_b_grad = self.acc_b_grad

        nabla_b, nabla_w = self.backprop(mini_batch[0].T,mini_batch[1].T)

        self.acc_w_grad = [
          self.momentum * vt - seta * nw
          for vt, nw in zip(acc_w_grad, nabla_w)
        ]
        self.acc_b_grad = [
          self.momentum * vt - seta * nb
          for vt, nb in zip(acc_b_grad, nabla_b)
        ]

        self.weights = [
          w + (1 + self.momentum) * v_next - self.momentum * v
          for w, v, v_next in zip(self.weights, acc_w_grad, self.acc_w_grad)
        ]
        self.biases = [
          b + (1 + self.momentum) * v_next - self.momentum * v
          for b, v, v_next in zip(self.biases, acc_b_grad, self.acc_b_grad)
        ]

        
    def backprop(self, x, y):
        # For a single input (x,y) return a pair of lists.
        # First contains gradients over biases, second over weights.
        g = x
        gs = [g] # list to store all the gs, layer by layer
        fs = [] # list to store all the fs, layer by layer
        for b, w in zip(self.biases, self.weights):
            f = np.dot(w, g)+b
            fs.append(f)
            g = sigmoid(f)
            gs.append(g)
        # backward pass <- both steps at once
        dLdg = self.cost_derivative(gs[-1], y)
        dLdfs = []
        for w,g in reversed(list(zip(self.weights,gs[1:]))):
            dLdf = np.multiply(dLdg,np.multiply(g,1-g))
            dLdfs.append(dLdf)
            dLdg = np.matmul(w.T, dLdf)
        
        dLdWs = [np.matmul(dLdf,g.T) for dLdf,g in zip(reversed(dLdfs),gs[:-1])] # automatic here
        dLdBs = [np.sum(dLdf,axis=1).reshape(dLdf.shape[0],1) for dLdf in reversed(dLdfs)] # CHANGE: Need to sum here
        return (dLdBs,dLdWs)

    def evaluate(self, test_data):
        # Count the number of correct answers for test_data
        pred = np.argmax(self.feedforward(test_data[0]),axis=0)
        corr = np.argmax(test_data[1],axis=1).T
        return np.mean(pred==corr)
    
    def cost_derivative(self, output_activations, y):
        return (output_activations-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None, lmbda=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                y_mini_batch = y_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                self.update_mini_batch((x_mini_batch, y_mini_batch), eta, lmbda)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate((x_test, y_test))))
            else:
                print("Epoch: {0}".format(j))


network = Network([784,30,10], 0.25)
network.SGD((x_train, y_train), epochs=200, mini_batch_size=100, eta=3.0, test_data=(x_test, y_test), lmbda=0.0015)



Epoch: 0, Accuracy: 0.6906
Epoch: 1, Accuracy: 0.862
Epoch: 2, Accuracy: 0.8959
Epoch: 3, Accuracy: 0.9068
Epoch: 4, Accuracy: 0.9123
Epoch: 5, Accuracy: 0.9171
Epoch: 6, Accuracy: 0.9224
Epoch: 7, Accuracy: 0.9258
Epoch: 8, Accuracy: 0.9287
Epoch: 9, Accuracy: 0.9309
Epoch: 10, Accuracy: 0.9334
Epoch: 11, Accuracy: 0.9357
Epoch: 12, Accuracy: 0.9365
Epoch: 13, Accuracy: 0.9378
Epoch: 14, Accuracy: 0.9396
Epoch: 15, Accuracy: 0.9401
Epoch: 16, Accuracy: 0.9412
Epoch: 17, Accuracy: 0.9422
Epoch: 18, Accuracy: 0.9424
Epoch: 19, Accuracy: 0.9438
Epoch: 20, Accuracy: 0.9451
Epoch: 21, Accuracy: 0.9457
Epoch: 22, Accuracy: 0.9465
Epoch: 23, Accuracy: 0.9466
Epoch: 24, Accuracy: 0.9473
Epoch: 25, Accuracy: 0.9476
Epoch: 26, Accuracy: 0.9481
Epoch: 27, Accuracy: 0.9483
Epoch: 28, Accuracy: 0.9488
Epoch: 29, Accuracy: 0.9498
Epoch: 30, Accuracy: 0.9499
Epoch: 31, Accuracy: 0.9507
Epoch: 32, Accuracy: 0.9509
Epoch: 33, Accuracy: 0.9522
Epoch: 34, Accuracy: 0.9522
Epoch: 35, Accuracy: 0.9528
Epo

In [146]:
# RMSProp + l1 regularization

class NetworkRMS(Network):

    @property
    def final_biases(self):
      return self.biases

    @property
    def final_weights(self):
      return self.weights
    
    def update_mini_batch(self, mini_batch, eta, lmbda=None):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch which is as in tensorflow API.
        # eta is the learning rate

        eps = 1e-8
        seta = eta/len(mini_batch[0])
        
        if lmbda is not None:
          self.l1_reg(seta * lmbda)

        nabla_b, nabla_w = self.backprop(mini_batch[0].T,mini_batch[1].T)

        self.acc_w_grad = [
          self.momentum * vt + (1-self.momentum) * (nw ** 2)
          for vt, nw in zip(self.acc_w_grad, nabla_w)
        ]
        self.acc_b_grad = [
          self.momentum * vt + (1-self.momentum) * (nb ** 2)
          for vt, nb in zip(self.acc_b_grad, nabla_b)
        ]

        self.weights = [
          w - seta * np.divide(nw, np.sqrt(v + eps))
          for w, v, nw in zip(self.weights, self.acc_w_grad, nabla_w)
        ]
        self.biases = [
          b - seta * np.divide(nb, np.sqrt(v + eps))
          for b, v, nb in zip(self.biases, self.acc_b_grad, nabla_b)
        ]


network2 = NetworkRMS([784,30,10], 0.9)
network2.SGD((x_train, y_train), epochs=200, mini_batch_size=100, eta=0.08, test_data=(x_test, y_test), lmbda=0.03)



Epoch: 0, Accuracy: 0.5956
Epoch: 1, Accuracy: 0.7551
Epoch: 2, Accuracy: 0.8463
Epoch: 3, Accuracy: 0.8737
Epoch: 4, Accuracy: 0.8867
Epoch: 5, Accuracy: 0.8959
Epoch: 6, Accuracy: 0.9031
Epoch: 7, Accuracy: 0.9077
Epoch: 8, Accuracy: 0.9131
Epoch: 9, Accuracy: 0.9164
Epoch: 10, Accuracy: 0.9199
Epoch: 11, Accuracy: 0.923
Epoch: 12, Accuracy: 0.9249
Epoch: 13, Accuracy: 0.9273
Epoch: 14, Accuracy: 0.9286
Epoch: 15, Accuracy: 0.93
Epoch: 16, Accuracy: 0.9319
Epoch: 17, Accuracy: 0.9333
Epoch: 18, Accuracy: 0.9344
Epoch: 19, Accuracy: 0.9354
Epoch: 20, Accuracy: 0.9369
Epoch: 21, Accuracy: 0.9376
Epoch: 22, Accuracy: 0.9387
Epoch: 23, Accuracy: 0.9394
Epoch: 24, Accuracy: 0.9401
Epoch: 25, Accuracy: 0.94
Epoch: 26, Accuracy: 0.9411
Epoch: 27, Accuracy: 0.9414
Epoch: 28, Accuracy: 0.9425
Epoch: 29, Accuracy: 0.943
Epoch: 30, Accuracy: 0.9433
Epoch: 31, Accuracy: 0.9437
Epoch: 32, Accuracy: 0.944
Epoch: 33, Accuracy: 0.9446
Epoch: 34, Accuracy: 0.945
Epoch: 35, Accuracy: 0.9454
Epoch: 36,

In [0]:
)